<a href="https://colab.research.google.com/github/Recommendation-System-Project/Recommendation-system/blob/main/IPS_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rnames = ['user_id','item_id','rating','timestamp']
rating = pd.read_table(r'ml-100k/u.data',names = rnames,engine = 'python')
rating = rating.set_index(['user_id','item_id'],drop=False).drop('timestamp',axis = 1)

In [ ]:
rating

,,user_id,item_id,rating
user_id,item_id,,,
196,242,196,242,3
186,302,186,302,3
22,377,22,377,1
244,51,244,51,2
166,346,166,346,1
...,...,...,...,...
880,476,880,476,3
716,204,716,204,5
276,1090,276,1090,1


## Matrix of Observed Rating

In [ ]:
rating_matrix = rating.pivot(index = 'user_id',columns = 'item_id',values = 'rating')
rating_matrix

item_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Matrix of Users Features

In [ ]:
unames = ['userid','age','gender','occupation','zip code']
users = pd.read_table(r'ml-100k/u.user',sep = '\|',names = unames, engine = 'python').drop('zip code', axis = 1)
users.head()

,userid,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# encode
num_feat_1 = ['age']
num_transformer_1 = Pipeline(steps=[
        ('imp', SimpleImputer(strategy='constant', fill_value=-1)),
    ])

num_feat_2 = ['gender']
num_transformer_2 = Pipeline(steps=[
        ('binarize', OrdinalEncoder()),
    ])

cat_feat = ['occupation']
cat_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])

preproc = ColumnTransformer(
transformers=[
            ('num_1', num_transformer_1, num_feat_1),
            ('num_2', num_transformer_2, num_feat_2),
            ('cat', cat_transformer, cat_feat)
        ])

pl = Pipeline(steps=[('preprocessor', preproc)])

In [ ]:
# First column Age, Second column gender, The rest is onehot occupation
user_features_matrix = pl.fit_transform(users)
user_features_matrix

array([[24.,  1.,  0., ...,  0.,  1.,  0.],
       [53.,  0.,  0., ...,  0.,  0.,  0.],
       [23.,  1.,  0., ...,  0.,  0.,  1.],
       ...,
       [20.,  1.,  0., ...,  1.,  0.,  0.],
       [48.,  0.,  0., ...,  0.,  0.,  0.],
       [22.,  1.,  0., ...,  1.,  0.,  0.]])

In [ ]:
user_occupation = user_features_matrix[:,2:]
user_occupation

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [ ]:
user_occupation.shape

(943, 21)

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
from tensorflow.keras.optimizers import SGD

X_train = user_occupation
 
## Encoder
encoder = Sequential()
# encoder.add(Flatten(input_shape=[21,]))
encoder.add(Dense(X_train.shape[1]))
encoder.add(Dense(400,activation="relu"))
encoder.add(Dense(200,activation="relu"))
encoder.add(Dense(100,activation="relu"))
encoder.add(Dense(50,activation="relu"))
encoder.add(Dense(5,activation="relu"))
 
## Decoder
decoder = Sequential()
decoder.add(Dense(50,input_shape=[5],activation='relu'))
decoder.add(Dense(100,activation='relu'))
decoder.add(Dense(200,activation='relu'))
decoder.add(Dense(400,activation='relu'))
decoder.add(Dense(21, activation="relu"))
# decoder.add(Reshape([943, 21]))
 
## Autoencoder
autoencoder = Sequential([encoder,decoder])
autoencoder.compile(loss="mse")
autoencoder.fit(X_train,X_train,epochs=50)
 
encoded_2dim = encoder.predict(X_train)
 
# AE = pd.DataFrame(encoded_2dim, columns = ['X1', 'X2'])
 
# AE['target'] = y_train
 
# sns.lmplot(x='X1', y='X2', data=AE, hue='target', fit_reg=False, size=10)

Epoch 1/50
30/30 [==============================] - 1s 5ms/step - loss: 0.0298
Epoch 2/50
30/30 [==============================] - 0s 6ms/step - loss: 0.0155
Epoch 3/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0106
Epoch 4/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0092
Epoch 5/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0091
Epoch 6/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0083
Epoch 7/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0084
Epoch 8/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0081
Epoch 9/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0081
Epoch 10/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0079
Epoch 11/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0080
Epoch 12/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0078
Epoch 13/50
30/30 [==============================

In [ ]:
encoded_2dim

array([[0.6610051 , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.29227167, 0.        , 0.        , 0.19521376],
       [0.2577684 , 0.16717583, 0.        , 0.12695386, 0.        ],
       ...,
       [0.27743307, 0.        , 0.        , 0.        , 0.3558712 ],
       [0.2007294 , 0.4974551 , 0.23575784, 0.        , 0.        ],
       [0.27743307, 0.        , 0.        , 0.        , 0.3558712 ]],
      dtype=float32)

## Matrix of Items Features

In [ ]:
inames = ['itemid','movie title','release date','video release date','IMDB URL','unknown','Action','Adventure'
          ,'Animation','Children\'s','Comedy','Crime','Documentrary','Drama','Fatasy','Film-Noir','Horror','Musical'
          ,'Mystery','Romance','Sci-Fi','Thriller','War','Western']
items = pd.read_table(r'ml-100k/u.item',sep = '\|',names = inames,engine = 'python',encoding='latin-1')
items.head()

,itemid,movie title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
items_matrix = items[['itemid', 'unknown', 'Action', 'Adventure', 'Animation',
                     'Children\'s', 'Comedy', 'Crime', 'Documentrary', 'Drama',
                     'Fatasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                     'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]
items_matrix.head()

,itemid,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentrary,Drama,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
